In [1]:
%load_ext autoreload
%autoreload 2


## Libraries

In [2]:
# Add root to path
import sys
import os
from pathlib import Path


sys.path.append("..")
from component.script.utilities.file_filter import (
    list_files_by_extension,
    filter_files_by_keywords,
)

from component.script.project import Project
from component.script.geo_utils import calculate_utm_rioxarray


## Load project

In [16]:
project_name = "nuevo2"

# Load the project from JSON
project = Project.load(project_name=project_name)


Project loaded from: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json
Loaded 17 processed variables


## Define base variable

The base variable will be uses as reference for all other variables in terms of resolution, crs and extent.

In [4]:
# First, we list all the raster files in the project directory
project.raw_variables

{'subj': LocalRasterVar(name='subj', data_type='raster', year=None, active=True, tags=[], path=PosixPath('/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data_raw/subj.tif'), raster_type='categorical', post_processing=[], default_crs=None, default_resolution=None),
 'protected_area': LocalRasterVar(name='protected_area', data_type='raster', year=None, active=True, tags=[], path=PosixPath('/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data_raw/protected_area.tif'), raster_type='categorical', post_processing=[], default_crs=None, default_resolution=None),
 'altitude': LocalRasterVar(name='altitude', data_type='raster', year=None, active=True, tags=[], path=PosixPath('/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data_raw/altitude.tif'), raster_type='continuous', post_processing=[], default_crs=None, default_resolution=None),
 'slope': LocalRasterVar(name='slope', data_type='raster', year=None, active=True, tags=[], path=PosixPath('/home/dguerrer

In [5]:
# We select the name of the variable we want to use

base_layer_name = "subj" # We can change this if our base has a different name
base_variable = project.raw_variables[base_layer_name]

In [6]:
user_defined_epsg_code = None

In [7]:
# If the user has not defined an EPSG code, we calculate it from the base variable
if user_defined_epsg_code is None:
    calculated_epsg = calculate_utm_rioxarray(
        base_variable.path
    )
    epsg_code = calculated_epsg
elif user_defined_epsg_code is not None:
    epsg_code = user_defined_epsg_code


In [8]:
# Reproject the base variable to the calculated or user-defined EPSG code
# This will be the reference for all other variables
reprojected_base = base_variable.reproject(target_epsg=epsg_code)

In [9]:
# We call a method to make this var the base of our project
reprojected_base.use_as_base_raster()

✓ Set 'subj_reprojected' as base raster for project 'nuevo2'
  Path: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data/subj_reprojected.tif
  CRS: EPSG:32618
  Resolution: 30
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


LocalRasterVar(name='subj_reprojected', data_type=<DataType.raster: 'raster'>, year=None, active=True, tags=[], path=PosixPath('/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data/subj_reprojected.tif'), raster_type='categorical', post_processing=[], default_crs='EPSG:32618', default_resolution=30)

## Calculate Forest Loss 

In [10]:
calculate_forest_loss = True
calculate_forest_loss_stack = True

In [11]:
from component.script.processing import get_forest_loss_calculated
from component.script.variables import LocalRasterVar


if calculate_forest_loss:

    forest_layers = project.filter_by_tags(["forest"], look_up_in="raw")
    forest_layers = [layer for layer in forest_layers.values()]
    
    forest_loss_layers = get_forest_loss_calculated(project, forest_layers)

    # now we add the new layers to the project
    for layer in forest_loss_layers:
        layer: LocalRasterVar
        # Raw because we have not projected it yet
        layer.add_as_raw()


Calculating forest loss between 2015 and 2020...
Calculating forest loss between 2015 and 2024...
Calculating forest loss between 2020 and 2024...
✓ Forest loss calculation complete!
✓ Added 'forest_loss_2015_2020' to raw variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json
✓ Added 'forest_loss_2015_2024' to raw variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json
✓ Added 'forest_loss_2020_2024' to raw variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


In [12]:
forest_loss_layers[0].year

2020

In [13]:
project.years

[2015, 2020, 2024]

In [14]:
from component.script.processing import generate_deforestation_raster


if calculate_forest_loss_stack:
    # We create a stack of the forest loss layers
    forest_loss_layers = project.filter_by_tags(["forest_loss"], look_up_in="raw")
    forest_loss_layers = [layer for layer in forest_loss_layers.values()]

    def_raster_var = generate_deforestation_raster(*forest_loss_layers, project)
    def_raster_var.add_as_raw()

✓ Added 'defostack' to raw variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


## Reproject and Rasterize Vector Data

In [15]:
project.reproject_all()


📍 Reprojecting 'subj'...
✓ Added 'subj_reprojected' to processed variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json

📍 Reprojecting 'protected_area'...
✓ Added 'protected_area_reprojected' to processed variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json

📍 Reprojecting 'altitude'...
✓ Added 'altitude_reprojected' to processed variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json

📍 Reprojecting 'slope'...
✓ Added 'slope_reprojected' to processed variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json

📍 Reprojecting 'forest_gfc_10_2015'...
✓ Added 'forest_gfc_10_2015_reprojected' to processed variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json

📍 Reprojecting 'forest_gfc_10_2020'...
✓ Added 'forest_gfc_10_2020

{'subj_reprojected': LocalRasterVar(name='subj_reprojected', data_type=<DataType.raster: 'raster'>, year=None, active=True, tags=[], path=PosixPath('/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data/subj_reprojected.tif'), raster_type='categorical', post_processing=[], default_crs='EPSG:32618', default_resolution=30),
 'protected_area_reprojected': LocalRasterVar(name='protected_area_reprojected', data_type=<DataType.raster: 'raster'>, year=None, active=True, tags=[], path=PosixPath('/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data/protected_area_reprojected.tif'), raster_type='categorical', post_processing=[], default_crs='EPSG:32618', default_resolution=30),
 'altitude_reprojected': LocalRasterVar(name='altitude_reprojected', data_type=<DataType.raster: 'raster'>, year=None, active=True, tags=[], path=PosixPath('/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data/altitude_reprojected.tif'), raster_type='continuous', post_processing=[], d

## Reproject Raster Data

## Calculate distance

In [19]:
project.processed_vars

{'subj_reprojected': LocalRasterVar(name='subj_reprojected', data_type='raster', year=None, active=True, tags=[], path=PosixPath('/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data/subj_reprojected.tif'), raster_type='categorical', post_processing=[], default_crs='EPSG:32618', default_resolution=30.0),
 'protected_area_reprojected': LocalRasterVar(name='protected_area_reprojected', data_type='raster', year=None, active=True, tags=[], path=PosixPath('/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data/protected_area_reprojected.tif'), raster_type='categorical', post_processing=[], default_crs='EPSG:32618', default_resolution=30.0),
 'altitude_reprojected': LocalRasterVar(name='altitude_reprojected', data_type='raster', year=None, active=True, tags=[], path=PosixPath('/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data/altitude_reprojected.tif'), raster_type='continuous', post_processing=[], default_crs='EPSG:32618', default_resolution=30.0),
 '

In [20]:
roads_projected_edge = (
    project.processed_vars["roads_reprojected"]
        .reproject(target_epsg=epsg_code)
        .apply_post_processing("edge")
        .add_as_processed()
)

✓ Added 'roads_reprojected_reprojected_edge' to processed variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


In [23]:
roads_projected_dist = (
    project.processed_vars["roads_reprojected"]
        .apply_post_processing("dist")
        .add_as_processed()
)

✓ Added 'roads_reprojected_dist' to processed variables
Project saved to: /home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/nuevo2_project.json


# Double check the dataset

In [25]:
project.processed_vars

{'subj_reprojected': LocalRasterVar(name='subj_reprojected', data_type='raster', year=None, active=True, tags=[], path=PosixPath('/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data/subj_reprojected.tif'), raster_type='categorical', post_processing=[], default_crs='EPSG:32618', default_resolution=30.0),
 'protected_area_reprojected': LocalRasterVar(name='protected_area_reprojected', data_type='raster', year=None, active=True, tags=[], path=PosixPath('/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data/protected_area_reprojected.tif'), raster_type='categorical', post_processing=[], default_crs='EPSG:32618', default_resolution=30.0),
 'altitude_reprojected': LocalRasterVar(name='altitude_reprojected', data_type='raster', year=None, active=True, tags=[], path=PosixPath('/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/nuevo2/data/altitude_reprojected.tif'), raster_type='continuous', post_processing=[], default_crs='EPSG:32618', default_resolution=30.0),
 '